___
### <center> <u> <h1> <strong> Utilisation de la bibliothèque SciPy </strong> </h1> </center>

<h3>
Dans ce notebook on verra comment utiliser la bibliothèque Scipy pour :
    
<ol>
    <li> Gérer des problèmes d'optimisation </li>
    <li> Interpoler des fonctions </li>
    <li> Faire un calcul d'intégration numérique </li>
    <li> Résoudre des équations différentielles </li>
    <li> Réaliser des transformée Fourier </li>
    <li> Resoudre des équation linéaires </li>
    <li> Résoudre des problèmes destatistique </li>

</ol>
</h3>
    
---

<h1> <strong> A - Importation des modules necessaires

In [ ]:
################################################################################
##################  Bibliothèque utilitaire mathémlatiques  ####################
################################################################################
import numpy as np
import scipy as sp
################################################################################
##################         Bibliothèque graphique           ####################
################################################################################
import matplotlib.pyplot as plt


<h1> <strong> B - Problème d'optimisation

<h1> <strong> B.1 - Simple problème de minimisation

<h3> <p> Supposons que nous soyons face à un problème dans lequel on a besoin e minimiser une grande donnée en Physique, Mahtémaque, Biologie . . </p>
    <p> On peut par exemple être comptable dans une entreprise donnée et on souhaite trouver le Mois sur les 10 dernière années où le chiffre d'affaire de l'entreprise a été au plus bas. Par ailleurs supposons que ce chifffre d'affaire soit modélisé par la fonction 

$f(t) = (t-3)^{2.6} + 5(t-3)^{5} $ </p>
    <p> Le problème se traduit donc à minimiser la fonction en question
   

In [ ]:
def f( t ) :
    return (t-3)**2.6 + 5*(t-3)**5

<h3> Pour se faire, la bibliothèque Scipy offre la possibilité de minimiser des fonctions grâce àla méthode $minimize$ de son module $optimize$

In [ ]:
from scipy.optimize import minimize

<h3> La documentation de la fonction peut être parcourue

In [ ]:
minimize?

<h3> Comme on peut le voir, pour trouver le minimum en  paramètres de base  il suffit de fournir à la fonction, l'indicatif de la fonction à minimiser et une valeur de départ.

In [ ]:
result = minimize( f , x0 = 3.1)

In [ ]:
result

In [ ]:
type(result)

<h3> La fonction retourne alors un objet qui contient toutes les informations necessaires sur la réponse au problème de minimisation <ul>  
    <li> $result.x$ donne la liste de valeurs de la fonction qui minimise cette dernière; résultat sous forme de list </li>
    <li> $result.fun$ donne la valeur minimale de la question </li>
    <li> $result.status$, $result.success$ et $result.message$ donnent des informations sur le statut de la requête de minimisation </li>
    </ul>

In [ ]:
result.x[0]

In [ ]:
result.fun

In [ ]:
result.success

In [ ]:
result.status

In [ ]:
result.message

<h3> Comment on pouvait s'y attendre :<ul> 
    <li> La fonction est minimale pour une valeur proche de 3 </li>
    <li> La valeur minimale de la fonction est proche de 0  </li>
    </ul>

<h1> <strong> B.1 - Problème de minimisation avec contraintes

<h3> <p>La même fonction pourrait être utilisée pour traiter un problème de minimisation imposant plusieurs contraintes sur les variables du problème.</p>
    <p>  Imaginons à présent que le chiffre d'affaire de l'entreprise soit plutot perçu comme une fonction qui ne dépende pas uniquement du nombre de ses employés, de la qualité du service ou des produits qu'elle fourni à la société et de son budget marketing. </p>
    <p>On notera $n$ le nombre d'enployé, $q$ la qualité du service ou des produits qu'elle fourni ou vend et $b$ son budget marketing </p>
    <p> Supposons ensuite que son chiffre d'affaire soit modélisable par la fonction analytique suivante     
    <p> $f(n, q, b) = 2.5n(b-60000-200q)^{2} - 2b(q-10+3n)^{2} +4b^{2}(7n-40-q)^{2} $ </p>
    <p> La question se pose alors de savoir quelle est le chiffre d'affaire minimile que l'entreprise pourrait espérer avoir avec moins de 100 employés, un budget de  inférieur à 10000 € et avec une contrainte supplémentaire qui soit que

$50np - b \geq 0$
    <p>  Toutes les contraintes du problèmes peuvent être resumées comme suit : </p>
    
$$100 - n \geq 0$$
$$10000 -b \geq 0 $$
$$50nq - b \geq 0$$
    <p>il faut aussi garder à l'esprit que $n$ et $b$ restent positifs et $q$ entre 0 et 100 </p>
    
$$n \geq 0$$
$$b \geq 0$$
$$0 \leq q \leq 100$$


- ### <strong> La fonction chiffre d'affaire

In [ ]:
def f ( x ) :
    n , q , b = x
    return 2.5*n*( b - 60000 -200*q)**2 -2*b*( q - 10 +3*n )**2 + 4*(b**2)*( 7*n - 40 -q)**2

- ### <strong> Les valeurs autorisées aux variables

In [ ]:
bornes = ((0, None), (0,20), (0, None))

- ### <strong> Les contraintes du problème

In [ ]:
contraintes =(
    {'type': 'ineq', 'fun': lambda x:  200 - x[0]  },
    {'type': 'ineq', 'fun': lambda x:  10000 - x[2]},
    {'type': 'ineq', 'fun': lambda x:  50*x[0]*x[1] - x[2]},
)

In [ ]:
resultat = minimize( f , (1 , 2 , 4000), constraints=contraintes, bounds=bornes, method = 'Powell')

In [ ]:
resultat

<h1> <strong> C - Problème d'interpolation

<h3> <p> Supposeez qu'en tant que statisticien, physicien, biologisteou ingénieur vous ayez pour mission de mesure une grandeur physique données qui est censée évoluer avec le temps. Au bout de 30 jours vous avez pu reccueillir un échantillon de 20 mesures qui se présentent comme suit </p>

In [ ]:
t_min, t_max = 0 , 30
N = 20


def mesure ( t ) : 
    base = 6*np.cos(2*np.pi*t*6/(t_max-t_min))*t**3.3/(1+  2*t + +t**3)
    bruit = np.random.rand(t.size) - np.random.rand(t.size)
    
    return base + bruit

t_sample = np.linspace(t_min , t_max, N)
y_sample = mesure ( t_sample)


plt.figure(figsize = (7,5))
plt.scatter(t_sample , y_sample, label= "Mesures obtenues")
plt.xlabel("Temps (Jours)")
plt.ylabel("Mesure ")
plt.legend(loc="upper left")
plt.show()

<h3> <p> Votre objectif à présent est de pouvoir retrouver la courbe générale de la fonction qui permet de retrouver ces valeurs mesurées si possible dans le but de pouvoir prédire des valeurs intermédiaire que vous avez peu être manqué le vintième ou le sixième jour de votre mesure peu être parceque vous aviez oublié, ou vous avez effectué la mesure vous vous l'avez perdu entre temps. </p>
    <p> Le module $interpolate$ de $scipy$ offre la fonction $interp1d$ qui permet de réaliser une telle tâche. </p>

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
interp1d?

<h1> <strong> C.1 - Interpolation linéaire

<h3> Dans un premier temps on va réaliser une interpolation linéaire qui consiste à joindre deux points adjacents de la mesure par une droite

In [ ]:
t = np.linspace( t_min , t_max , 200)

In [ ]:
linear_interpolate = interp1d( t_sample, y_sample, kind='linear')
y_linear = linear_interpolate( t )

In [ ]:
plt.figure(figsize=(8,5))

plt.scatter(t_sample , y_sample , label='mesures')
plt.plot(t , y_linear , label = 'Lin interpol')

plt.legend(loc="upper left")
plt.ylim(y_linear.min(), y_linear.max()+1)
plt.xlim(t.min(), t.max()+5)
plt.show()

<h3> Comme on peut le voir la forme de la courbe d'interpolation ne corresponds pas visuellement à ce qu'on pourrait considérer comme une fonction analytique car cette dernière est une fonction continues par morceauux qui est définie sur plusieurs sous domaines séparéments; d'où la necessité d'une méthode d'interpolation plus adaptée

<h1> <strong> C.2 - Interpolation cubique

In [ ]:
cubic_interpolate = interp1d( t_sample, y_sample, kind='cubic')
y_cubic = cubic_interpolate( t )

In [ ]:
plt.figure(figsize=(8,5))

plt.scatter(t_sample , y_sample , label='mesures')
plt.plot(t , y_cubic , label = 'cubic interpol')
plt.plot(t , y_linear , label = 'Lin interpol')

plt.legend(loc="upper left")
plt.ylim(y_cubic.min(), y_cubic.max()+2)
plt.xlim(t.min(), t.max()+5)
plt.show()

<h3> Avec une interpolation cubique on a une courbe qui est un peu plus lisse que celle avec une interpolation simplement linéaire. D'autres méthodes peuvent être utilisées

<h1> <strong> C.3 - Interpolation quadratique

In [ ]:
quad_interpolate = interp1d( t_sample, y_sample, kind='quadratic')
y_quad = quad_interpolate( t )

In [ ]:
plt.figure(figsize=(8,5))

plt.scatter(t_sample , y_sample , label='mesures')
plt.plot(t , y_cubic , label = 'cubic interpol')
plt.plot(t , y_quad , label = 'quad interpol')

plt.legend(loc="upper left")
plt.ylim(y_quad.min(), y_quad.max())
plt.xlim(t.min(), t.max())
plt.show()

<h3> On peut ainsi utiliser l'objet renvoyé par la méthde $interp1d$ pour obtenir des valeurs intermédiaires qu'on n'a pas pu mesurer dans la pratique

In [ ]:
t = 20

In [ ]:
linear_interpolate(t)

In [ ]:
cubic_interpolate(t)

In [ ]:
quad_interpolate(t)

<h1> <strong> D - Problème de modélisation ( fitting )

<h3> <p> La position du problème ici est la suivante : on est dans le même cas de figure que dans le problème d'interpolation mais l'object est de trouver l'expression analytique d'un fonction qui s'approche le plus possible de des valeurs discrète obtenues par expérience </p>

---

- ## <strong> Authors </strong>

 #### <ul> <p> - <strong><a href="https://www.linkedin.com/in/brice-kengni-zanguim/" target="_blank">Brice KENGNI ZANGUIM :</a> </strong></p> <ul> Artificial Intelligence Engeneer at home and holds a PhD in Nano-physic. His research focused on using <strong>Machine Learning, Deep Learning, Signal Processing, Natural langage Processing, Computer Vision and so one</strong> to figure out how the world behave and impact human cognition. Brice KENGNI ZANGUIM has been working at home on mastering his own AI projects in many domains while seeking for a position in a company. </ul></ul>


---

- ## <strong> Other Contributors </strong>

#### <ul>None</ul>

---

- ## <strong> Change Log </strong>

|  Date (DD-MM-YYYY) |  Version | Changed By  |  Change Description |  Purpose  |
|---|---|---|---|---|
| 19- 03 - 2023 | 1.0 | Brice KENGNI ZANGUIM | Updated the file | Mem,save |
|-|-|-|-|
|   |   |   |   |   |

### <h3 align="center"> © BKZ Corporation 2023. All rights reserved. <h3/>
---